# 🚀 이진 분류 모델 (데이터 정렬 문제 해결)

## 📊 클래스 불균형 문제 해결:
## 1. 3클래스 → 2클래스로 단순화
## 2. 임계값 0.2%로 낮춤
## 3. 클래스 가중치 적용
## 4. 데이터 정렬 문제 해결

In [ ]:
# 📦 필요한 라이브러리 설치
%pip install pandas==2.0.3 numpy==1.24.3 scikit-learn==1.3.0 xgboost==1.7.6 lightgbm==4.0.0 tensorflow==2.13.0 pandas-ta==0.3.14b0 supabase

In [ ]:
!pip install --upgrade pip!pip install xgboost lightgbm pandas-ta supabase plotly numba# 🔧 NumPy 2.0 호환성 패치 (즉시 해결!)import numpy as npif not hasattr(np, 'NaN'):    np.NaN = np.nan    print("✅ NumPy 패치 적용 완료!")# 이제 pandas_ta 정상 동작import pandas_ta as taprint("✅ pandas_ta 임포트 성공!")print(f"버전: {ta.version}")# 📚 라이브러리 임포트import pandas as pdfrom sklearn.ensemble import RandomForestClassifierfrom sklearn.model_selection import train_test_split, cross_val_scorefrom sklearn.metrics import accuracy_score, classification_report, confusion_matriximport lightgbm as lgbfrom supabase import create_clientimport warningswarnings.filterwarnings('ignore')print('✅ 라이브러리 로드 완료')

In [ ]:
# 🔗 Supabase 연결
SUPABASE_URL = "https://your-project.supabase.co"
SUPABASE_KEY = "your-anon-key"

supabase = create_client(SUPABASE_URL, SUPABASE_KEY)
print('✅ Supabase 연결 완료')

In [ ]:
# 📊 데이터 수집 (배치 처리)
def collect_data_in_batches(symbol='ADAUSDT', timeframe='1m', target_records=500000):
    """배치로 데이터 수집"""
    all_data = []
    offset = 0
    batch_size = 1000
    
    print(f'🔄 {symbol} {timeframe} 데이터 수집 시작...')
    
    while len(all_data) < target_records:
        try:
            response = supabase.table('crypto_ohlcv').select('*').eq('symbol', symbol).eq('timeframe', timeframe).order('timestamp', desc=True).range(offset, offset + batch_size - 1).execute()
            
            if not response.data:
                break
                
            all_data.extend(response.data)
            offset += batch_size
            
            if len(all_data) % 10000 == 0:
                print(f'📊 수집된 데이터: {len(all_data):,}개')
                
        except Exception as e:
            print(f'❌ 배치 수집 오류: {e}')
            break
    
    print(f'✅ 데이터 수집 완료: {len(all_data):,}개')
    return all_data

# 데이터 수집 실행
raw_data = collect_data_in_batches()

# DataFrame 생성
df = pd.DataFrame(raw_data)
df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms')
df = df.sort_values('datetime')

print(f'📅 기간: {df["datetime"].min()} ~ {df["datetime"].max()}')
print(f'📊 총 데이터: {len(df):,}개')

In [ ]:
# 📈 기술적 지표 계산
def calculate_technical_indicators(df):
    """기술적 지표 계산"""
    # 이동평균
    df['sma_5'] = ta.sma(df['close'], length=5)
    df['sma_10'] = ta.sma(df['close'], length=10)
    df['sma_20'] = ta.sma(df['close'], length=20)
    df['ema_9'] = ta.ema(df['close'], length=9)
    df['ema_21'] = ta.ema(df['close'], length=21)
    
    # MACD
    macd = ta.macd(df['close'])
    df['macd'] = macd['MACD_12_26_9']
    df['macd_signal'] = macd['MACDs_12_26_9']
    df['macd_histogram'] = macd['MACDh_12_26_9']
    
    # RSI
    df['rsi'] = ta.rsi(df['close'], length=14)
    
    # 볼린저 밴드
    bb = ta.bbands(df['close'])
    df['bb_upper'] = bb['BBU_20_2.0']
    df['bb_lower'] = bb['BBL_20_2.0']
    df['bb_width'] = (df['bb_upper'] - df['bb_lower']) / df['close']
    df['bb_position'] = (df['close'] - df['bb_lower']) / (df['bb_upper'] - df['bb_lower'])
    
    # 가격 변화율
    df['price_change'] = df['close'].pct_change()
    df['price_change_5'] = df['close'].pct_change(5)
    df['price_change_10'] = df['close'].pct_change(10)
    
    # 거래량 지표
    df['volume_sma'] = ta.sma(df['volume'], length=20)
    df['volume_ratio'] = df['volume'] / df['volume_sma']
    
    # 시간 특성
    df['hour'] = df['datetime'].dt.hour
    df['day_of_week'] = df['datetime'].dt.dayofweek
    df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)
    
    return df

# 지표 계산
df = calculate_technical_indicators(df)
print('✅ 기술적 지표 계산 완료')

In [ ]:
# 🎯 이진 분류 목표 변수 생성
def create_binary_target(df, lookforward=5, threshold=0.002):  # 0.2% 임계값
    # 미래 가격 변화율 계산
    df['future_return'] = df['close'].shift(-lookforward) / df['close'] - 1
    
    # 이진 분류: 상승(1) vs 하락(0)
    df['target'] = np.where(df['future_return'] > threshold, 1, 0)
    
    return df

# 목표 변수 생성
df = create_binary_target(df)
print('✅ 이진 분류 목표 변수 생성 완료')
print(f'📊 타겟 분포:\n{df["target"].value_counts()}')
print(f'📈 상승 비율: {df["target"].mean()*100:.1f}%')

In [ ]:
# 🔧 데이터 정렬 및 전처리 (중요!)
feature_columns = [
    'sma_5', 'sma_10', 'sma_20', 'ema_9', 'ema_21',
    'macd', 'macd_signal', 'macd_histogram',
    'rsi',
    'bb_upper', 'bb_lower', 'bb_width', 'bb_position',
    'price_change', 'price_change_5', 'price_change_10',
    'volume_sma', 'volume_ratio',
    'hour', 'day_of_week', 'is_weekend'
]

# 모든 특성과 타겟이 있는 행만 선택
all_columns = feature_columns + ['target']
df_clean = df[all_columns].dropna()

# 특성과 타겟 분리
X = df_clean[feature_columns]
y = df_clean['target']

print(f'🔧 데이터 정렬 완료')
print(f'📊 특성 데이터: {X.shape}')
print(f'📊 타겟 데이터: {y.shape}')
print(f'✅ 샘플 수 일치: {X.shape[0] == y.shape[0]}')

In [ ]:
# 🚀 이진 분류 LightGBM 모델
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report
import pickle
import json

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 클래스 가중치 계산
class_weights = {0: 1, 1: 5}  # 상승에 5배 가중치

# LightGBM 모델 설정
model = lgb.LGBMClassifier(
    n_estimators=1000,
    learning_rate=0.05,
    max_depth=8,
    num_leaves=31,
    random_state=42,
    n_jobs=-1,
    verbose=-1,
    class_weight=class_weights
)

print('🚀 이진 분류 LightGBM 모델 훈련 시작...')
print(f'📊 훈련 데이터: {X_train.shape}')
print(f'📊 테스트 데이터: {X_test.shape}')
print(f'⚖️ 클래스 가중치: {class_weights}')

# 모델 훈련
model.fit(X_train, y_train)

# 예측
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f'✅ 모델 훈련 완료')
print(f'📊 정확도: {accuracy:.4f}')

In [ ]:
# 📊 모델 성능 평가
from sklearn.model_selection import cross_val_score

# 교차 검증
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(model, X, y, cv=cv, scoring='accuracy')

print('📊 교차 검증 결과:')
print(f'평균 정확도: {cv_scores.mean():.4f} (±{cv_scores.std():.4f})')
print(f'개별 정확도: {cv_scores}')

# 분류 리포트
print('\n📋 분류 리포트:')
print(classification_report(y_test, y_pred))

# 혼동 행렬
print('\n📊 혼동 행렬:')
cm = confusion_matrix(y_test, y_pred)
print(cm)

# 특성 중요도
feature_importance = pd.DataFrame({
    'feature': feature_columns,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

print('\n🏆 상위 10개 중요 특성:')
print(feature_importance.head(10))